In [9]:
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import math
import numpy as np 

# A function which trains itself to predict whether a cancer is metastatic or not by using aneural netowrk. 
def neural_network(): 
    
    ##### (1) Load and prepare data. 
    data =  datasets.load_breast_cancer()
    x, y = data.data, data.target
    
    # Get data dimensions. 
    _, num_features = x.shape
    
    # Split the data into training data and testing data.
    x_training, x_testing, y_training, y_testing = train_test_split(x, y, test_size=0.33, random_state=1234)

    # Scale the data. 
    x_training = StandardScaler().fit_transform(x_training)
    x_testing = StandardScaler().fit_transform(x_testing)
    
    # Convert data to tensors.
    x_training = torch.from_numpy(x_training.astype(np.float32))
    x_testing = torch.from_numpy(x_testing.astype(np.float32))
    
    num_samples = y_training.shape
    y_training = torch.from_numpy(y_training.reshape(num_samples[0], 1))
    y_training = y_training.type(torch.float32)
                                 
    num_samples = y_testing.shape
    y_testing = torch.from_numpy(y_testing.reshape(num_samples[0], 1))
    y_testing = y_testing.type(torch.float32)     

    ##### (2) Create our model. 
    class NeuralNetwork(nn.Module): 
        def __init__(self, num_features): 
            super(NeuralNetwork, self).__init__()
            self.linear_1 = nn.Linear(num_features, math.floor(num_features/2))
            self.linear_2 = nn.Linear(math.floor(num_features/2), math.floor(num_features/4))
            self.linear_3 = nn.Linear(math.floor(num_features/4), 1)

            self.sigmoid = nn.Sigmoid()
            
        def forward(self, x):
            output_1 = self.sigmoid(self.linear_1(x))
            output_2 = self.sigmoid(self.linear_2(output_1))
            y_predicted = self.sigmoid(self.linear_3(output_2))
            return y_predicted 
        
    # Create an instance of our model. 
    model = NeuralNetwork(num_features)
    
    ##### (3) Establish the loss and the optimiser. 
    calc_loss = nn.BCELoss() # Use built in binary cross entropy loss function from PyTorch.
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # We're using stochastic gradient descent. 
   
    ##### (4) Training loop. 
    num_epochs = 10000
    loss_array = []
    for epoch in range(num_epochs):
    
        # Forward pass: compute the output of the layers given the input data
        y_predicted = model(x_training)
        loss = calc_loss(y_predicted, y_training)
        
        # Log the loss per epoch.
        loss_value = loss.detach().numpy()
        loss_value = loss_value.item()
        loss_array.append(loss_value)
        
        # Backward pass: compute the output error with respect to the expected output and then go backward into the network and update the weights using gradient descent.
        loss.backward()
        
        # Update the weights.
        optimizer.step()

        # Zero out the gradients. 
        optimizer.zero_grad()